In [335]:
%matplotlib inline

In [336]:
import ipywidgets as widgets
from IPython.display import display
from plasmapy.particles import Particle
import astropy.units as units
from inspect import signature
import importlib
import json

In [337]:
values_container = dict()
def create_label(label):
    return widgets.Label(label)
class GenericWidget:
    def __init__(self, property_name,property_alias="",values_cont=values_container):
        self.property_name = property_name
        self.property_alias = property_alias or property_name
        self.widget = None
        self.values_cont = values_cont
        self.unit = None
    def set_unit(self,unit):
        self.unit = unit
    def get_widget(self):
        return self.widget
    def set_place_holder(self,text):
        self.widget.placeholder = text
    def create_widget(self):
        raise NotImplementedError()
    def post_creation(self):
        self.set_place_holder("")
        self.widget.observe(self.handle_change,names="value")
    def edge_case(self,value):
        pass
    def edge_case_condition(self,value):
        return False
    def try_change_value(self,value):
        self.values_cont[self.property_name]=value
    def display_error(self,value):
        if self.widget:
            self.widget.layout.border="2px solid red"
            self.widget.description = "Invalid "+self.property_alias
    def convert_to_unit(self,change):
        if self.unit:
            return change.new*self.unit
        return change.new
    def handle_change(self,change):
        value = self.convert_to_unit(change)
        if self.edge_case_condition(value):
            self.edge_case(value)
        else:
            try:
                self.try_change_value(value)
            except:
                self.display_error(value)

class FloatBox(GenericWidget):
    def __init__(self, property_name,min=-1e50,max=1e50):
        super().__init__(property_name)
        self.min = min
        self.max = max
    def create_widget(self,style={"description_width": "initial"}):
        self.widget = widgets.BoundedFloatText(name=self.property_name,min=self.min, 
            max=self.max, value=0, step=0.1,style=style)
        self.post_creation()

class ParticleBox(GenericWidget):
    def __init__(self, property_name,property_alias=None):
        super().__init__(property_name,property_alias=property_alias)

    def edge_case_condition(self, value):
        return value is None or value == ""
    def edge_case(self, value):
        self.values_cont[self.property_name] = None
        self.widget.description = ""
        self.widget.layout.border = "1px solid black"
    def try_change_value(self, value):
        ion = Particle(value)
        self.values_cont[self.property_name] = ion
        self.widget.layout.border="1px solid black"
        self.widget.description = ""

    def create_widget(self,style={"description_width": "initial"}):
        self.widget = widgets.Text(style=style)
        self.post_creation()
        self.set_place_holder("Enter ion type particle for e.g. p,He 2+")

def create_widget(widget_type,**kwargs):
    unit = None
    if 'unit' in kwargs:
        unit = kwargs['unit']
        del kwargs['unit']
    widget_element = widget_type(**kwargs)
    widget_element.create_widget()
    if unit:
        widget_element.set_unit(unit)
    return widget_element.get_widget()
    

In [338]:
def colored(r, g, b, text):
    return "\033[38;2;{};{};{}m{} \033[38;2;255;255;255m".format(r, g, b, text)
class FunctionInfo:
    def __init__(self, module_name, function_name, values_cont=values_container):
        self.module = module_name
        self.fname = function_name
        self.fattr = getattr(importlib.import_module(module_name),function_name)
        self.values_cont = values_cont
        self.spec_combo = None
        self.sig = list(signature(self.fattr).parameters.keys())
        self.output_widget = widgets.Output()
    def add_combo(self,spec_combo):
        if not self.spec_combo:
            self.spec_combo = []
        self.spec_combo.append(spec_combo)
    def get_output_widget(self):
        return self.output_widget
    def produce_arg(self, spec):
        args_dict = dict()
        for arg in spec:
            if arg in self.values_cont and self.values_cont[arg] is not None:
                args_dict[arg] = self.values_cont[arg]

        return args_dict

    def error_message(self,spec):
        print(colored(0,0,0,"["),end="")
        for arg in spec:
            if arg in self.values_cont and self.values_cont[arg] is not None:
                print(colored(0,128,0,arg+":present,"),end="")
            else:
                print(colored(255,0,0,arg+":missing,"),end="")
        print(colored(0,0,0,"]"))
    def process(self):
        self.output_widget.clear_output()
        args_dict = dict()
        if self.spec_combo:
           for spec in self.spec_combo:
               args_dict = self.produce_arg(spec)
               if len(args_dict) == len(spec):
                   break
        else:
            args_dict = self.produce_arg(self.sig)
        with self.output_widget:
            try:
                self.output_widget.layout.border="0px"
                print(self.fname+" : "+str(self.fattr(**args_dict)))
            except Exception as e:
                self.output_widget.layout.border="1px solid red"
                print(self.fname+" : could not be computed one or more parameter is missing - check below for missing parameters")
                if self.spec_combo:
                    for spec in self.spec_combo:
                        self.error_message(spec)
                else:
                    self.error_message(self.sig)


In [339]:
with open('properties_metadata.json') as f:
    data = json.load(f)

process_queue = []
app = widgets.Tab()
children = []
for i,title in enumerate(data):
    grid_layout = widgets.GridspecLayout(10,2,width="100%")
    for j,prop in enumerate(data[title]):
        fn = FunctionInfo(prop["module_name"],prop["function_name"])
        if "spec_combo" in prop:
            for spec_combo in prop["spec_combo"]:
                fn.add_combo(spec_combo)
        grid_layout[j,0] = create_label(prop["function_name"]+":")
        grid_layout[j,1] = fn.get_output_widget()
        process_queue.append(fn)
    children.append(grid_layout)
    app.set_title(i,title)
app.children = children

In [340]:
def create_button():
    button = widgets.Button(description="Calculate Properties",button_style="info")
    return button

def handle_button_click(event):
    for fn in process_queue:
        fn.process()

In [341]:
grid = widgets.GridspecLayout(10, 2,width="700px")
grid.layout.margin="10px"
## Row 0 - Magnetic Field
grid[0,0] = create_label("B - Magnetic Field Magnitude(T):")
grid[0,1] = create_widget(FloatBox,property_name="B",unit=units.T)
## Row 1 - Density
grid[1,0] = create_label("\u03C1 - Density(kg/m3):")
grid[1,1] = create_widget(FloatBox,property_name="density",min=0,unit=units.kg/units.m**3)
## Row 2 - Particle Type
grid[2,0] = create_label("Particle(Ion):")
grid[2,1] = create_widget(ParticleBox,property_name="particle",property_alias="ion_type")

calculate_button = widgets.Button(description="Calculate Properties",button_style="info")
calculate_button.on_click(handle_button_click)
grid[-1,0] = calculate_button
display(grid)

GridspecLayout(children=(Label(value='B - Magnetic Field Magnitude(T):', layout=Layout(grid_area='widget001'))…

In [342]:
app